# 黑色金属 - 铁矿石

In [1]:
# 设置工作目录
import os
os.getcwd()
if os.name == 'posix':
    os.chdir('/Volumes/Repository/Projects/ffa/')
else:
    os.chdir("E:\\Document\\Project\\ffa")

In [2]:
# 加载依赖模块
import pandas as pd
import akshare as ak
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date
import importlib
import commodity
import json

In [22]:
# 重新加载salary计算模块
importlib.reload(commodity)

<module 'commodity' from 'E:\\Document\\Project\\ffa\\commodity.py'>

## 数据准备与预处理

In [3]:
symbol_id = 'I'
symbol_name = '铁矿石'
fBasePath = 'steel/data/up-stream/铁矿石/'
json_file = './steel/setting.json'

### 数据索引设置

In [7]:
# 品种数据索引初始化
# 首次使用json配置文件存取品种的数据索引
data_index = {'主力合约': {'Name': "dominant_contract", 'Source':'AKShare', 'Path': fBasePath + '铁矿石基差.xlsx', 'Field':'dominant_contract', 'DataFrame': "df_basis"},
                '近月合约': {'Name': "near_contract", 'Source':'AKShare', 'Path': fBasePath + '铁矿石基差.xlsx', 'Field':'near_contract', 'DataFrame': "df_basis"},
                '主力合约收盘价': {'Name': "dominant_close_price", 'Source':'Choice', 'Path': fBasePath + '铁矿石期货价格.xlsx', 'Field':'期货收盘价(主力):铁矿石', 'DataFrame': "df_dominant"},
                '主力合约结算价': {'Name': "dominant_settle_price", 'Source':'Choice', 'Path': fBasePath + '铁矿石期货价格.xlsx', 'Field':'期货结算价(主力):铁矿石', 'DataFrame': "df_dominant"},
                '现货价格': {'Name': "spot_price", 'Source':'AKShare', 'Path': fBasePath + '铁矿石基差.xlsx', 'Field':'spot_price', 'DataFrame': "df_basis"},
                '基差': {'Name': "basis", 'Source':'AKShare', 'Path': fBasePath + '铁矿石基差.xlsx', 'Field':'dom_basis', 'DataFrame': "df_basis"},
                '基差率': {'Name': "basis_rate", 'Source':'AKShare', 'Path': fBasePath + '铁矿石基差.xlsx', 'Field':'dom_basis_rate', 'DataFrame': "df_basis"},
                '产量': {'Name': "production", 'Source':'Choice', 'Path': fBasePath + '铁矿石产量.xlsx', 'Field':'产量:铁矿石原矿:当月值', 'DataFrame': "df_prodoction"},
                #'销量': {'Name': "sales", 'Source':'Choice', 'Path': fBasePath + '螺纹钢销量.xlsx', 'Field':'销量：钢筋：累计值', 'DataFrame': "df_sales"},
                '库存': {'Name': "instock", 'Source':'Choice', 'Path': fBasePath + '铁矿石库存.xlsx', 'Field':'国内铁矿石港口库存量', 'DataFrame': "df_instock"},
                '库存消费比': {'Name': "instock_consumption_ratio", 'Source':'Choice', 'Path': fBasePath + '铁矿石库存.xlsx', 'Field':'全国:进口铁矿石:库存消费比', 'DataFrame': "df_instock_consumption_ratio"},
                # TODO: 校验Choice提供的另外一套库存和仓单数据
                '仓单': {'Name': "receipt", 'Source':'Choice', 'Path': fBasePath + '铁矿石库存.xlsx', 'Field':'仓单数量:铁矿石', 'DataFrame': "df_instock"}
                }
symbol_setting = {'DataIndex': data_index,
                  '利润公式': {'Name': 'profit_formula', 'X': 1.0, 'Y': 1.0, '其他成本': 0}}
# TOOD: 先读取已有配置文件内容后，与写入内容合并后再保存
with open(json_file, 'w', encoding='utf-8') as config_file:
    try:
        json.dump({symbol_name: symbol_setting}, config_file, indent=4, ensure_ascii=False)
    except IOError as e:
        print(f"Error saving configuration: {e}")  


In [8]:
symbol_data = commodity.SymbolData(symbol_id, symbol_name, json_file)
# df = symbol_data.update_akshare_file(mode='initial', start_date='20230101', end_date='20231130')

In [9]:
# 构造品种数据访问对象
symbol_data = commodity.SymbolData(symbol_id, symbol_name, json_file)
merged_data = symbol_data.merge_data()

### 更新数据

- 通过AKshare接口更新现货价格数据
- 手动更新Choice导出数据（使用Microsoft Excel打开，自动更新后保存）

In [ ]:
# 更新现货价格数据 - AKShare
# 追加最新数据
df_basis = symbol_data.update_akshare_file(mode='append')
# 2011、2012年数据待更新
# symbol_data.update_akshare_file(mode='period', start_date='20120101', end_date='20121231')

## 基差分析

### 基差走势

In [ ]:
# 加载基差数据（奇货可居网）
df_rb_basis = pd.read_csv('data/rb-main-basis-20231028.csv')
df_rb_basis['基差'] = df_rb_basis['现货'] - df_rb_basis['期货']
# 加载库存数据（AK-Share网站）
futures_inventory_em_df = ak.futures_inventory_em(symbol="螺纹钢")
futures_inventory_em_df['日期'] = pd.to_datetime(futures_inventory_em_df['日期'], format="%Y-%m-%d")
# 统一日期格式
df_rb_basis['日期'] = pd.to_datetime(df_rb_basis['日期'], format="%Y-%m-%d")
# 合并基差和库存数据
df_rb0 = pd.merge(df_rb_basis, futures_inventory_em_df, on='日期', how='outer')
# 空数据清理
df_rb0 = df_rb0.fillna(0)
# 为基差率（正负值）标记颜色
df_rb0['基差率颜色'] = df_rb0['基差率'] > 0
df_rb0['基差率颜色'] = df_rb0['基差率颜色'].replace({True:1, False:0})
max_y = df_rb0['期货'].max() * 1.05
min_y = df_rb0['期货'].min() * 0.95

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
                   vertical_spacing=0.01, subplot_titles=('基差分析', '基差率', '库存'), 
                   row_width=[0.15, 0.15, 0.7])

#fig_main = make_subplots(specs=[[{"secondary_y": True}]])
fig_future_price = go.Scatter(x=df_rb0['日期'], y=df_rb0['期货'], name='期货价格', marker_color='rgb(84,134,240)')
fig_spot_price = go.Scatter(x=df_rb0['日期'], y=df_rb0['现货'], name='现货价格', marker_color='rgb(105,206,159)')
fig_basis = go.Scatter(x=df_rb0['日期'], y=df_rb0['基差'], stackgroup='one', name='基差', marker_color='rgb(239,181,59)')
#fig_main.add_trace(fig_future_price)
#fig_main.add_trace(fig_basis_rate, secondary_y=True)

fig.add_trace(fig_basis, secondary_y=True)
fig.add_trace(fig_future_price, row = 1, col = 1)
fig.add_trace(fig_spot_price, row = 1, col = 1)

fig_basis_rate = go.Bar(x=df_rb0['日期'], y = df_rb0['基差率'], name='基差率', marker_color='rgb(244,128,68)')
#fig_basis_rate = go.Bar()
#fig_basis_rate.mode = 'markers'
# fig_basis_rate.x = df_rb0['日期']
# fig_basis_rate.y = df_rb0['基差率']
fig_basis_rate.marker.colorscale = ['green', 'red']
fig_basis_rate.marker.color = df_rb0['基差率颜色']
#fig_basis_rate.marker.size = 2
fig_basis_rate.marker.showscale = False
fig_basis_rate.showlegend = False

fig.add_trace(fig_basis_rate, row = 2, col = 1)
fig_storage = go.Bar(x=df_rb0['日期'], y=df_rb0['库存'], name='库存', marker_color='rgb(234,69,70)')
#fig_storage = px.bar(df_rb0, x='日期', y='库存')
fig.add_trace(fig_storage, row = 3, col = 1)

trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
dt_all = pd.date_range(start=df_rb0['日期'].iloc[0],end=df_rb0['日期'].iloc[-1])
dt_all = [d.strftime("%Y-%m-%d") for d in dt_all]
dt_breaks = list(set(dt_all) - set(trade_date))

# X轴坐标按照年-月显示
fig.update_xaxes(
    showgrid=True,
    zeroline=True,
    showline=True,
    dtick="M1",  # 按月显示
    ticklabelmode="period",   # instant  period
    tickformat="%b\n%Y",
    rangebreaks=[dict(values=dt_breaks)],
    rangeslider_visible = False, # 下方滑动条缩放
    rangeselector = dict(
        # 增加固定范围选择
        buttons = list([
            dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
            dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
            dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
            dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
            dict(step = 'all')
            ]))
)
#fig.update_traces(xbins_size="M1")
fig.update_layout(
    yaxis_range=[min_y,max_y],
    #autosize=False,
    #width=800,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.show()

## 基差率分析

### 基差率季节分析

In [ ]:
df_rb0['年度'] = df_rb0['日期'].dt.year
df_rb0['年内日期'] = df_rb0['日期'].dt.strftime('1900-%m-%d')
fig_basis_rate_season = px.line(df_rb0,
                                x='年内日期',
                                y='基差率',
                                color='年度',
                                #color_discrete_sequence=px.colors.qualitative.G10)
                                color_discrete_sequence=['lightgray', 'lightblue', 'orange', 'red'])
fig_basis_rate_season.update_layout(
    title={
        'text':'基差率季节分析',
        'xanchor':'center'},
    margin=dict(l=10, r=10, t=40, b=10)
)

fig_basis_rate_season.show()

In [ ]:
df1 = hd.load_choice_file('data/钢铁产业链/中游数据/螺纹钢/螺纹钢库存.xlsx')
df1

### 基差率月度涨跌统计

### 基差率频率分布

## 跨期分析

### 期限结构

In [ ]:
# 加载合约基础数据
futures_comm_info = pd.read_excel('data/common_info.xlsx')
spec_contact_list = futures_comm_info[futures_comm_info.合约名称.str.startswith('螺纹钢')]
fig_term = make_subplots(specs=[[{"secondary_y": True}]])
fig_term.add_trace(go.Scatter(x=spec_contact_list['合约代码'], y=spec_contact_list['现价']))
# 获取最新现货价格
spot_price = df_rb0[df_rb0['现货']!=0]['现货'].iloc[-1]
fig_term.add_hline(y=spot_price)
fig_term.update_layout(
    title={
        'text':'期限结构'
    },
    #autosize=False,
    width=800,
    #height=800,
    margin=dict(l=10, r=10, t=40, b=10)
)
fig_term.show()

### 套利分析

#### 价差分析-多期排列

#### 价差分析-跨期价差矩阵

#### 基差-月差分析

#### 价差季节性分析

## 库存

### 库存数据

In [ ]:
futures_inventory_em_df = ak.futures_inventory_em(symbol="螺纹钢")
futures_inventory_em_df['日期'] = pd.to_datetime(futures_inventory_em_df['日期'], format="%Y-%m-%d")
df_rb_basis['日期'] = pd.to_datetime(df_rb_basis['日期'], format="%Y-%m-%d")
df_rb0 = pd.merge(df_rb_basis, futures_inventory_em_df, on='日期', how='outer')

### 库存周期

### 库存效费比

### 仓单

#### 注册仓单

In [ ]:
#future_receipt = ak.get_receipt(start_day="20230101", end_day="20231031", vars_list=["RB"])
#future_receipt

#### 仓单日报

#### 期转现

#### 交割统计

## 利润

### 现货利润

### 期货盘面利润

### 利润期限结构

## 综合分析

### 基差-库存-利润分析

### 基差-月差分析

### 期限结构-库存/仓单分析